# Code for Ice-Cube 3D CNN

- Oct 29, 2018: This code just makes plots for previously trained CNNs

In [4]:
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
import glob
import pickle
import time

In [5]:
%matplotlib widget
# %matplotlib inline

Useful blog for keras conv3D: http://learnandshare645.blogspot.com/2016/06/3d-cnn-in-keras-action-recognition.html

In [6]:
# keras modules
import keras
from keras import layers, models, optimizers, callbacks  # or tensorflow.keras as keras
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc, roc_auc_score
from keras.models import load_model



Using TensorFlow backend.


## Modules

In [11]:
def f_get_ydata_and_wts(data_dir,f1,f2):
    ''' Load extracted data from files. Just extracting ydata and weights
    returns : inpy,weights as arrays
    '''

    inpy=np.load(data_dir+f1+'.npy')
    wts=np.load(data_dir+f2+'.npy')
    
    return(inpy,wts)
    
def f_shuffle_data(inpy,wts):
    ## Shuffle data
    
    # Setting seed
    seed=243
    np.random.seed(seed=seed)

    ## Get shuffled array of indices
    shuffle_arr=np.arange(inpy.shape[0])
    np.random.shuffle(shuffle_arr)
    inpy=inpy[shuffle_arr]
    wts=wts[shuffle_arr]

    return inpy,wts

    
def f_plot_learning(history):
    
    fig=plt.figure()
    # Plot training & validation accuracy values
    fig.add_subplot(2,1,1)
    plt.plot(history['acc'],label='Train')
    plt.plot(history['val_acc'],label='Validation')
#     plt.title('Model accuracy')
    plt.ylabel('Accuracy')

    # Plot loss values
    fig.add_subplot(2,1,2)
    plt.plot(history['loss'],label='Train')
    plt.plot(history['val_loss'],label='Validation')
#     plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='best')


def f_plot_roc_curve(fpr,tpr):
    '''
    Module for roc plot and printing AUC
    '''
    plt.figure()
    # plt.plot(fpr,tpr)
    plt.scatter(fpr,tpr)
    plt.semilogx(fpr, tpr)
  # Zooms
    plt.xlim([10**-7,1.0])
    plt.ylim([0,1.0])
#     plt.xscale('log')
#     plt.xlim(1e-10,1e-5)
    plt.show()

    # AUC 
    auc_val = auc(fpr, tpr)
    print("AUC: ",auc_val)


def f_plot_fit(inpy,wts,model_dict):
    '''
    Plot fit results.
    '''
    
#     model_save_dir='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/saved_models/'
    model_save_dir=model_loc
    model_name=model_dict['name'] # string for the model
    fname_model,fname_history='model_{0}.h5'.format(model_name),'history_{0}.pickle'.format(model_name)
    
        
    ########################
    ### Read model and history
    
    ### Check if files exist
    assert os.path.exists(model_save_dir+fname_model),"Model not saved"
    assert os.path.exists(model_save_dir+fname_history),"History not saved"
    
    model=load_model(model_save_dir+fname_model)
    with open(model_save_dir+fname_history,'rb') as f:
        history= pickle.load(f)
    
    ########################
    model.summary()
    # Plot tested model
    f_plot_learning(history)
    
    ########################
    # Get test predictions
    
    test_file_name=model_save_dir+'y-predict_model-'+str(model_name)+'.pred'
    test_y_file_name=model_save_dir+'y-test_model-'+str(model_name)+'.test'
    test_weights_file_name=model_save_dir+'wts-test_model-'+str(model_name)+'.test'    
    print("Using test prediction from previous test",test_file_name)

    assert os.path.exists(test_file_name),"y-predictions not saved"
    y_pred=np.loadtxt(test_file_name)
    ydata=np.loadtxt(test_y_file_name)
    wts=np.loadtxt(test_weights_file_name)
    assert(test_y.shape==y_pred.shape),"Data %s and prediction arrays %s are not of the same size"%(test_y.shape,y_pred.shape)
    
    fpr,tpr,threshold=roc_curve(ydata,y_pred,sample_weight=wts)
    print(fpr.shape,tpr.shape,threshold.shape)
    f_plot_roc_curve(fpr,tpr)
    
    model_dict['model'],model_dict['history']=model,history
    
    return model_dict



In [12]:
if __name__=='__main__':
    ###Extract data : Only extract y-data and weights for tests.
    data_dir='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/extracted_data_v/data/temp/'
#     f1,f2='processed_input_regular_y','processed_input_regular_wts'
#     inpy,wts=f_get_ydata_and_wts(data_dir,f1,f2)

    f1,f2='processed_input_reserved_y','processed_input_reserved_wts'
    inpy,wts=f_get_ydata_and_wts(data_dir,f1,f2)
    test_y,test_wts=inpy[:],wts[:]   
    
    
    ## Shuffle data
#     inpy,wts=f_shuffle_data(inpy,wts)
    ###Split data
#     size=inpy.shape[0]
#     train_idx=int(size*0.75)
#     test_y,test_wts=inpy[train_idx:],wts[train_idx:]


In [13]:
print(inpy.shape,wts.shape,test_y.shape,test_wts.shape)

(752604,) (752604,) (752604,) (752604,)


In [14]:
model_loc='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/Nov12_saved_models_reserved_data/'
dict_list=[]
for i in range(1,6):
# for i in range(1,2):
    model_dict={'name':str(i),'description':None,'model':None,'history':None}
    print(i,model_dict)
    model_dict=f_plot_fit(test_y,test_wts,model_dict)
    dict_list.append(model_dict)

1 {'name': '1', 'description': None, 'model': None, 'history': None}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 20, 60, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 10, 20, 60, 10)    280       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 10, 30, 10)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 5, 10, 30, 10)     2710      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 2, 5, 15, 10)      0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 2, 5, 15, 10)      2710      
_________________________________________________________________
max_poo

FigureCanvasNbAgg()

Using test prediction from previous test /global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/Nov12_saved_models_reserved_data/y-predict_model-1.pred
(722206,) (722206,) (722206,)


FigureCanvasNbAgg()

AUC:  0.9414992093460843
2 {'name': '2', 'description': None, 'model': None, 'history': None}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 20, 60, 1)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 10, 20, 60, 10)    280       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 5, 10, 30, 10)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 10, 30, 10)     0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 5, 10, 30, 10)     2710      
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 2, 5, 15, 10)      0         
________________________________________________

...

AUC:  0.8992825149094021
5 {'name': '5', 'description': None, 'model': None, 'history': None}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10, 20, 60, 1)     0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 10, 20, 60, 6)     726       
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 3, 6, 20, 6)       0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 3, 6, 20, 6)       0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 3, 6, 20, 6)       4326      
_________________________________________________________________
max_pooling3d_14 (MaxPooling (None, 1, 2, 6, 6)        0         
________________________________________________

FigureCanvasNbAgg()

Using test prediction from previous test /global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/Nov12_saved_models_reserved_data/y-predict_model-5.pred
(188381,) (188381,) (188381,)


FigureCanvasNbAgg()

AUC:  0.9047911546846832


### Comparing models

In [ ]:
# dict_list[0]

In [ ]:
## Comparing different models:

for md in dict_list:
#     print(md)
    hist=md['history']
#     print(hist)
#     print(md)
    print('Model %s'%(md['name']))
    for key in hist.keys():
        print(key,hist[key])
#     print(md[''])
        

## -----------------------------------------------

### Questions:
- Why are fpr and tpr different for 2 different models?


#### Notes:
- model.fit 
    - batch_size= sample of data used for training (subset of full training set). 
    - epoch= number of runs over training data
    - callbacks=
    
- for layers.Input need size (x,y,z,1) in channels_last mode.

#### Roc curve notes:
- We know y-value depending on signal or background (0 or 1).
- The 3D-Cnn gives us a prediction for y, as a float between 0 or 1.
- We must use a cut (threshold) to determine what constitues 0 / 1. Eg. 0.5
- This gives us a false +ve rate a, true +ve .(fpr and tpr)
- Roc curve plots this when varying the threshold
- AUC gives area under this curve.

In [ ]:
# Plotting weights
print(train_wts.shape,test_wts.shape)

# Train data 
plt.figure()
plt.plot(train_wts)
plt.title("train + cv data weigts ")
plt.show()

plt.figure()
plt.plot(test_wts)
plt.title("test data weights")
plt.show()



## View created figures

In [ ]:
# from IPython.display import IFrame


# model_loc='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/Nov5_saved_models/'
# fle='learning_model1.'
# fname=model_loc+"%s.pdf"%(fle)
# fname='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/Nov5_saved_models/learning_model1.eps'
# print(fname)
# print(os.path.exists(fname))
# IFrame(fname, width=600, height=300)

In [ ]:
# from IPython.display import Image
# Image(fname)

## To do
- pick the best model
- test on reserve data set
- running with multiple cores on a batch node.
- using multiple nodes
- using GPU nodes
- Test a host of models using ipyparallel
- make changes to incorporate regular data in training and reserved data in testing
- way to store tested values for easy plotting

## Notes:
- Code to 
    - show model summary and plots.
    - just view existing plots in files.

In [ ]:
Time taken for data extraction
# regular
print("Regular data set")
t1=20*113 #seconds 
t2=104*80 
t3_concat=323 
t4_total=175*60
print(t1,t2,t3_concat,t4_total)
print("total time in hours",t4_total/3600)

# reserved
print("Reserved data set")
t1=76*113 #seconds 
t2=570*90 
t3_concat=0
t4_total=t1+t2
print(t1,t2,t3_concat,t4_total)
print("total time in hours",t4_total/3600)
